# Introduction

This Python 3 notebook explores the exported VMWare inventory generated by [RVTools](https://www.robware.net/) and generates various statistical analyses of the data for assessing the scale, complexity, and other characteristics.

The analysis is directed towards understanding the feasibility of migrating these VMWare virtual machines to Red Hat's OpenShift Virtualization Platform.

# Initial Setup

This section configures the script and the required packages.

### Required Packages

This script uses `pandas` and `numpy` for the analysis.

In [ ]:
# Analyze the VM inventory generated by rvtools
import pandas as pd
import os
from pathlib import Path
import matplotlib.pyplot as plt

### Configuration

Set up a few options and other configuration parameters.

In [ ]:
# Set the display option to prevent line wrapping
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

### Set up the data directory
First, set up the directory where the RVTools Excel data files will be stored. This folder must also contain an index.xlsx file (refer to the provided template for the expected format). The index.xlsx file should list the valid RVTools Excel file names along with their corresponding vCenter instances. This script assumes that there is one Excel file per vCenter instance.

In [ ]:
# Get the current script directory
current_dir = "."

# Specify the directory containing the Excel files
DATA_DIR = os.path.join(current_dir, '../data')

In [ ]:
# Constants and configuration parameters

# The index file
INDEX_FILENAME = "index.xlsx"
INDEX_FILEPATH = os.path.join(DATA_DIR, INDEX_FILENAME)
INDEX_SHEETNAME = "index"

### Functions

This section defines a few functions for decomposing the analysis code.

In [ ]:
def read_rvtools_excel_files(directory, filenames_to_process):
    """
    Reads multiple Excel files exported from RVtools software and returns a dictionary
    with filenames as keys and a dictionary of DataFrames (one for each sheet) as values.

    The `index.xlsx` and `index_template.xlsx` files are explicitly ignored.

    Parameters:
    directory (str): The directory containing the Excel files.

    filenames_to_process (list): A list of filenames to process.

    Returns:
    dict: A dictionary containing the DataFrames from each Excel file.
    """
    rvtools_data = {}

    for filename in os.listdir(directory):
        # Only process files listed in the index file.
        filename_base, _ = os.path.splitext(filename)
        if filename_base not in filenames_to_process: continue
        if filename in ['index.xlsx', 'index_template.xlsx']: continue

        if filename.endswith('.xlsx') or filename.endswith('.xls'):
            filepath = os.path.join(directory, filename)
            excel_data = pd.read_excel(filepath, sheet_name=None)
            rvtools_data[filename] = excel_data

    return rvtools_data

# Read, Clean and Filter the RVTools data

This section reads the `RVTools` exported files. It uses an `index.xlsx` metadata file to identify the
in-scope `vCenter` instances for the migration.

### Read the index metadata file

First read the index file to determine the RVTools files to process. This file contains additional metadata,
including the `vCenter` instances in scope, which can be customized to process specific instances.

In [ ]:
# First read the index Excel file
import pandas as pd

# Read the index Excel file
index_df = pd.read_excel(
    INDEX_FILEPATH, sheet_name=INDEX_SHEETNAME, 
    nrows=19, index_col='vCenter', 
    true_values=['Yes', 'Y'], false_values=['No', 'N'], 
    na_filter=False, dtype={'In Scope': bool}
)

# Clean up column names and handle missing values
index_df.columns = index_df.columns.str.replace(' ', '_')
index_df.fillna('', inplace=True)

# Filter for in-scope vCenters
inscope_df = index_df[index_df['In_Scope']].reset_index()

# Count occurrences of each vCenter
pivot_table = inscope_df.groupby('vCenter').size().reset_index(name='Count')

# Print in a structured table format
print("\nIn-Scope vCenter Instances:")
print(pivot_table.to_string(index=False))

### Read the RVTool Exported Spreadsheets

Read all the RVTools exported files available in the `data` directory. The data will be read into a dictionary,
with one entry per file, where the key is the filename, and the value is **another** nested dictionary with the spreadsheet's
sheet name as the key, and a `DataFrame` containing the sheets values.

The files **need** to be in the Microsoft `xlsx` format.

**Note**: This step will take a few minutes to complete. Be patient!

In [ ]:
import pandas as pd
import os

# Read the index Excel file
index_df = pd.read_excel(
    INDEX_FILEPATH, sheet_name=INDEX_SHEETNAME, 
    nrows=19, index_col='vCenter', 
    true_values=['Yes', 'Y'], false_values=['No', 'N'], 
    na_filter=False, dtype={'In Scope': bool}
)

# Clean up column names and handle missing values
index_df.columns = index_df.columns.str.replace(' ', '_')
index_df.fillna('', inplace=True)

# Ensure 'In_Scope' column is boolean
index_df['In_Scope'] = index_df['In_Scope'].astype(bool)

# Filter for in-scope vCenters
inscope_df = index_df[index_df['In_Scope']].reset_index()

# Extract list of in-scope vCenter instances
inscope_vcenter_instances = inscope_df['vCenter'].tolist()

# Function to read RVTools Excel files while excluding 'index_template.xlsx' and 'index.xlsx'
def read_rvtools_excel_files(directory, vcenters):
    rvtools_data = {}
    exclude_files = {"index_template.xlsx", "index.xlsx"}  # Set of filenames to exclude

    for file in os.listdir(directory):
        if file.endswith(".xlsx") and file.lower() not in exclude_files:
            file_path = os.path.join(directory, file)
            try:
                xls = pd.ExcelFile(file_path)
                sheets = {sheet: xls.parse(sheet) for sheet in xls.sheet_names}
                rvtools_data[file] = sheets
            except Exception as e:
                print(f"Error reading {file}: {e}")
    
    return rvtools_data

# Read the RVTools Excel files
rvtools_data = read_rvtools_excel_files("../data", inscope_vcenter_instances)

# Display the loaded data (for demonstration purposes)
for filename, sheets in rvtools_data.items():
    print(f'Processed RVTools File: {filename}')
    for sheet_name, df in sheets.items():
        print(f"  Sheet: {sheet_name} Shape: {df.shape}")

print(f'Total files processed: {len(rvtools_data)}')

### Create the consolidated data frames

Create two consolidated dataframes containing information from all `vCenter` instances:

1. One dataframe containing the `vInfo` details
2. The second one containing the `vHost` details

These will be used later to summarize the information.

In [ ]:
# Assuming rvtools_data is already populated with the sheets data

import pandas as pd

# Initialize the lists
vinfo_sheets = []
vhost_sheets = []

# Load vInfo and vHost sheets into the lists
for filename, sheets in rvtools_data.items():
    vCenter = filename.split('.')[0].lower()  # Extract vCenter instance name from the filename

    # Ensure 'vInfo' and 'vHost' sheets exist before processing
    if 'vInfo' in sheets and 'vHost' in sheets:
        # Add the vCenter instance name to each sheet
        sheets['vInfo']['vCenter'] = vCenter
        sheets['vHost']['vCenter'] = vCenter

        # Append to the respective lists
        vinfo_sheets.append(sheets['vInfo'])
        vhost_sheets.append(sheets['vHost'])

# Filter out empty or all-NA DataFrames
vinfo_sheets = [df for df in vinfo_sheets if not df.dropna(how='all').empty]
vhost_sheets = [df for df in vhost_sheets if not df.dropna(how='all').empty]

# Concatenate the filtered DataFrames only if there are valid sheets
consolidated_vinfo_df = pd.concat(vinfo_sheets, ignore_index=True) if vinfo_sheets else pd.DataFrame()
consolidated_vhost_df = pd.concat(vhost_sheets, ignore_index=True) if vhost_sheets else pd.DataFrame()

# Verify data ingestion success with improved readability
print("\n✅ Data Ingestion Verification ✅")
print(f"Total vInfo records: {len(consolidated_vinfo_df)}")
print(f"Total vHost records: {len(consolidated_vhost_df)}\n")

if not consolidated_vinfo_df.empty:
    print("🔹 Sample vInfo Data:")
    print(consolidated_vinfo_df.head().to_string(index=False), "\n")

### Distribution of VMs per In-Scope vCenter

In [ ]:
import matplotlib.pyplot as plt

# Group by vCenter and count VMs
grouped_summary = consolidated_vinfo_df.groupby("vCenter").size().reset_index(name="Count")

# Print summary
total_vms = grouped_summary["Count"].sum()
total_vcenters = grouped_summary.shape[0]

print(f"Overall Distribution of {total_vms:,} VMs in the {total_vcenters:,} vCenter instances:\n")
print(grouped_summary)
print("\nNote: This is the OVERALL VM count and may include VM templates and SRM placeholders")

# Plot a pie chart
plt.figure(figsize=(8, 8))
plt.pie(
    grouped_summary["Count"],
    labels=grouped_summary["vCenter"],
    autopct="%1.1f%%",
    startangle=140,
)
plt.title("Distribution of ALL VMs by vCenter Instances")
plt.show()

### Clean the VM List
Filter the consolidated `vInfo` content using the following criteria:

1. Remove all `template` entries
2. Remove all `SRM Placeholder` entries
3. Remove all `orphaned VM object` entries
4. Remove all `other objects`

In [ ]:
import re
import pandas as pd
from IPython.display import display

# Define patterns to ignore in the VM column
ignore_patterns = ["virtual_appliance", "virtual appliance", "CTX"]

# Define OS types to filter out
os_filter_patterns = [
    "Microsoft Windows 10 (64-bit)", "AlmaLinux (64-bit)", "CentOS 4/5 (64-bit)", "panos",
    "CentOS 4/5/6/7 (64bit-)", "CentOS 7 (64-bit)", "CentOS 8 (64-bit)", "Debian GNU/Linux 11 (64-bit)",
    "Other (32-bit)", "Other (64-bit)", "Other 3.x Linux (64-bit)", "Other 3.x or later Linux (64-bit)",
    "Other Linux (64-bit)", "SUSE Linux Enterprise 11 (64-bit)", "SUSE Linux Enterprise 12 (64-bit)",
    "SUSE Linux Enterprise 15 (64-bit)", "Ubuntu Linux (64-bit)", "VMware Photon OS (64-bit)",
]

# Normalize the VM column for consistent matching
if 'VM' in consolidated_vinfo_df.columns:
    consolidated_vinfo_df['VM'] = consolidated_vinfo_df['VM'].astype(str).str.strip().str.lower()

# Normalize ignore patterns for VM filtering
ignore_patterns_lower = [pattern.lower() for pattern in ignore_patterns]
regex_vm_ignore = '|'.join([re.escape(pattern) for pattern in ignore_patterns_lower])

# Normalize OS filter patterns for OS filtering
os_filter_patterns_lower = [os_type.lower() for os_type in os_filter_patterns]

# Check if 'Total disk capacity MiB' exists, if not, default to 1 (to prevent exclusion)
if 'Total disk capacity MiB' in consolidated_vinfo_df.columns:
    consolidated_vinfo_df['Total disk capacity MiB'] = consolidated_vinfo_df['Total disk capacity MiB'].fillna(0)
else:
    print("⚠️ Warning: 'Total disk capacity MiB' column is missing! Defaulting to non-zero values.")
    consolidated_vinfo_df['Total disk capacity MiB'] = 1  # Prevents exclusion

# Define filter conditions
filter_condition = (
    # Ensure VM does not match ignore patterns
    (~consolidated_vinfo_df.get('VM', pd.Series('')).str.contains(regex_vm_ignore, case=False, na=False)) &
    # Ensure OS columns do not match os_filter_patterns
    (~consolidated_vinfo_df.get('OS according to the VMware Tools', pd.Series('')).str.lower().isin(os_filter_patterns_lower)) &
    (~consolidated_vinfo_df.get('OS according to the configuration file', pd.Series('')).str.lower().isin(os_filter_patterns_lower)) &
    # Exclude templates
    (consolidated_vinfo_df.get('Template', pd.Series(False)) == False) &
    # Exclude SRM placeholders
    (consolidated_vinfo_df.get('SRM Placeholder', pd.Series(False)) == False) &
    # Exclude orphaned VMs
    (consolidated_vinfo_df.get('Connection state', pd.Series('')) != 'orphaned') &
    # Exclude powered-off VMs
    (consolidated_vinfo_df.get('Powerstate', pd.Series('')) != 'poweredOff') &
    # Exclude VMs with zero disk capacity
    (consolidated_vinfo_df['Total disk capacity MiB'] != 0)
)

# Apply filter to get "in-scope" VMs
filtered_vinfo_df = consolidated_vinfo_df[filter_condition]
ignored_vm_artifacts = len(consolidated_vinfo_df) - len(filtered_vinfo_df)

# Display results
print("\n✅ **Filtering Summary** ✅")
print(f"🔹 Removed: {ignored_vm_artifacts:,} templates, SRM placeholders, orphaned, powered-off VMs, and excluded patterns.")
print(f"🔹 Filtered (In-Scope) VM count: {len(filtered_vinfo_df):,}.\n")

# Display the list of "in-scope" VMs after filtering
if not filtered_vinfo_df.empty:
    print("\n🔍 In-Scope of Filtered VMs:")
    display(filtered_vinfo_df[['VM', 'OS according to the VMware Tools', 'OS according to the configuration file']].head())

# Debug: Verify that ignore patterns and OS filters were respected
print("\n🔍 Out-of-Scope Filtered VMs")
print(filtered_vinfo_df[['VM', 'OS according to the VMware Tools', 'OS according to the configuration file']].drop_duplicates().head())

### Further filter down the In-Scope List
Filter down to the in-scope subset of the `vCenter` instances for further analysis.

This selection of the scoped instances is provided by the `index.xlsx` metadata file.

In [ ]:
import pandas as pd

# Select the in-scope vCenter instances
inscope_vinfo_condition = filtered_vinfo_df['vCenter'].isin(inscope_vcenter_instances)
inscope_vinfo_df = filtered_vinfo_df[inscope_vinfo_condition]

# Summary stats for in-scope VMs
inscope_vm_count = len(inscope_vinfo_df)
percent_inscope_vms = (inscope_vm_count / len(filtered_vinfo_df)) * 100.0

print("\n✅ VM Scope Summary")
print(f"🔹 {inscope_vm_count:,} VMs are in-scope ({percent_inscope_vms:0.2f}% of {len(filtered_vinfo_df):,} total VMs).\n")

# Create a pivot table for VMs by vCenter
vm_pivot = inscope_vinfo_df.pivot_table(index="vCenter", aggfunc="size").reset_index()
vm_pivot.columns = ["vCenter", "VM Count"]
display(vm_pivot)

# Select the in-scope hosts
inscope_vhost_condition = consolidated_vhost_df['vCenter'].isin(inscope_vcenter_instances)
inscope_vhost_df = consolidated_vhost_df[inscope_vhost_condition]

# Summary stats for in-scope hosts
inscope_host_count = len(inscope_vhost_df)
percent_inscope_hosts = (inscope_host_count / len(consolidated_vhost_df)) * 100.0

print("\n✅ Host Scope Summary")
print(f"🔹 {inscope_host_count:,} hosts are in-scope ({percent_inscope_hosts:0.2f}% of {len(consolidated_vhost_df):,} total hosts).\n")

# Create a pivot table for hosts by vCenter
host_pivot = inscope_vhost_df.pivot_table(index="vCenter", aggfunc="size").reset_index()
host_pivot.columns = ["vCenter", "Host Count"]
display(host_pivot)

# Perform the Analysis

The primary analysis begins from this section. We compute the following:

1. A consolidated summary of _all_ `vCenter` instances (including the out of scope instances)
2. A count of the **in-scope** guest operating systems
3. A grouping of the disk usage by tiers, for the **in-scope** VMs

### 1- Create a consolidated view of the VM landscape

This is a pivot table centered on the vCenter instances.

In [ ]:
# Create a pivot table for the VM info
# Create a pivot table for the VM info with filtered in-scope VMs
vinfo_pivot_df = filtered_vinfo_df.pivot_table(
    index='vCenter',
    values=['VM', 'CPUs', 'Memory', 'NICs', 'Total disk capacity MiB'],
    aggfunc={
        'VM': 'count',
        'CPUs': 'sum',
        'Memory': 'sum',
        'NICs': 'sum',
        'Total disk capacity MiB': 'sum'
    },
    margins=False
)

# Create a pivot table for in-scope VM count only
in_scope_vm_count_df = filtered_vinfo_df.pivot_table(
    index='vCenter',
    values='VM',
    aggfunc='count'
).rename(columns={'VM': 'In-Scope VM Count'})

# Host pivot table for additional host info
vhost_pivot_df = consolidated_vhost_df.pivot_table(
    index='vCenter',
    values=['Host', '# VMs total', '# CPU', '# Cores'],
    aggfunc={
        'Host': 'count',
        '# VMs total': 'sum',
        '# CPU': 'sum',
        '# Cores': 'sum'
    },
    margins=False
)

# Join tables and include the in-scope VM count
consolidated_summary_df = vinfo_pivot_df.join(vhost_pivot_df).join(in_scope_vm_count_df, how='left')
consolidated_summary_df['In-Scope VM Count'] = consolidated_summary_df['In-Scope VM Count'].fillna(0).astype(int)

# Calculate column totals
consolidated_summary_totals = consolidated_summary_df.sum(axis=0).to_frame().T
consolidated_summary_totals.index = ['Total']

# Append totals row to the consolidated summary
consolidated_summary_with_totals = pd.concat([consolidated_summary_df, consolidated_summary_totals])
consolidated_summary_with_totals.index.name = 'vCenter'

# Display the final summary with in-scope VM count
consolidated_summary_with_totals


### 2- Summarize the Operating Systems

In this section, we summarize the guest operating systems for the **in-scope** `vCenter` instances.

In [ ]:
# Summarize the Operating Systems and list by count.
import matplotlib.pyplot as plt
import numpy as np

# Create a combined OS column
inscope_vinfo_df['Final OS'] = np.where(
    inscope_vinfo_df['OS according to the VMware Tools'].notnull() & 
    (inscope_vinfo_df['OS according to the VMware Tools'].str.strip() != ''),
    inscope_vinfo_df['OS according to the VMware Tools'].str.strip(),
    inscope_vinfo_df['OS according to the configuration file'].str.strip()
)

# Ensure the 'VM' column exists and has valid data
if 'VM' not in inscope_vinfo_df.columns:
    raise KeyError("'VM' column is missing from the DataFrame.")

# Create the pivot table and sort by VM count in descending order
inscope_guest_os_pivot = inscope_vinfo_df.pivot_table(index='Final OS', values='VM', aggfunc='count')
inscope_guest_os_pivot = inscope_guest_os_pivot.sort_values(by='VM', ascending=True)  # Ascending for bar chart

# Check if pivot table is empty
if inscope_guest_os_pivot.empty:
    raise ValueError("The pivot table is empty. Check your input data.")

# Calculate percentages
total_vms = inscope_guest_os_pivot['VM'].sum()
percentages = (inscope_guest_os_pivot['VM'] / total_vms * 100).round(1)

# Create figure and axis
fig, ax = plt.subplots(figsize=(12, 8))

# Create horizontal bar chart
bars = ax.barh(inscope_guest_os_pivot.index, inscope_guest_os_pivot['VM'], color=plt.cm.tab10.colors)

# Add labels to bars
for bar, count, percentage in zip(bars, inscope_guest_os_pivot['VM'], percentages):
    ax.text(bar.get_width() + 2, bar.get_y() + bar.get_height()/2, 
            f"{count} VMs ({percentage}%)", va='center', fontsize=8, color='black')

# Style and labels
ax.set_xlabel("Number of VMs", fontsize=10)
ax.set_ylabel("Operating Systems", fontsize=10)
ax.set_title("Guest OS Distribution for In-Scope VMs", fontsize=12, pad=20)
ax.grid(axis='x', linestyle='--', alpha=0.7)  # Add grid lines for readability

# Adjust layout
plt.tight_layout()
plt.show()

### 3- OSs with over 500 VMs Associated

In [ ]:
# Filter and display OSs with over 500 VMs
over_500_vms = inscope_guest_os_pivot[inscope_guest_os_pivot['VM'] > 500]
display(over_500_vms)

# Check if there are any OSs with more than 500 VMs
if over_500_vms.empty:
    print("No operating systems have more than 500 VMs.")
else:
    # Pie chart for OSs with over 500 VMs
    plt.figure(figsize=(10, 8))  # Slightly wider for better readability

    # Define a color palette
    colors = plt.cm.tab10.colors[:len(over_500_vms)]

    # Generate pie chart
    wedges, texts, autotexts = plt.pie(
        over_500_vms['VM'],
        labels=over_500_vms.index,
        autopct='%1.1f%%',
        startangle=140,
        colors=colors,
        wedgeprops={'edgecolor': 'black', 'linewidth': 1}  # Add borders for clarity
    )

    # Improve text readability
    for text in texts:
        text.set_fontsize(10)  # Adjust label size
    for autotext in autotexts:
        autotext.set_fontsize(10)  # Adjust percentage text size
        autotext.set_color('white')  # Improve visibility

    # Add title
    plt.title('Operating Systems with Over 500 VMs', fontsize=12, pad=20)

    # Adjust layout to prevent labels from overlapping
    plt.tight_layout()
    plt.show()



### 4- Categorize Host Compute Nodes

Categorize ALL the compute nodes by the model number

In [ ]:
# Group all hosts by model
all_host_model_pivot_df = consolidated_vhost_df.pivot_table(index=['Vendor', 'Model'], values='Host', aggfunc='count')
display(all_host_model_pivot_df)

# Bar chart for all host models
plt.figure(figsize=(12, 8))  # Make it wider to avoid label overlap

# Create a bar chart
bar_plot = all_host_model_pivot_df['Host'].plot(kind='bar', color=plt.cm.tab10.colors[:len(all_host_model_pivot_df)], edgecolor='black', figsize=(12, 8))

# Add title and labels
plt.title('All Host Node Compute Models', fontsize=12)
plt.xlabel('Host Model', fontsize=10)
plt.ylabel('Host Count', fontsize=10)

# Improve x-tick readability by rotating the labels
plt.xticks(rotation=90, ha='right', fontsize=8)

# Show the bar chart
plt.tight_layout()  # Adjust layout to prevent overlapping labels
plt.show()

### 5- Group the VMs by disk-size tier

This section groups the **in-scope** VMs into categories defined by allocated disk-size tiers.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Convert disk size to TB and categorize into tiers
mib_to_tb_conversion_factor = 2**20 / 10**12
inscope_vinfo_df['Disk Size TB'] = inscope_vinfo_df['Total disk capacity MiB'] * mib_to_tb_conversion_factor

# Define disk size bins and labels
disk_size_bins = [0, 2, 10, 20, 40, 100, float('inf')]
disk_bin_labels = ['Easy (<=2 TB)', 'Medium (<=10 TB)', 'Hard (<=20 TB)', 'Very Hard (<=40 TB)', 'Extremely Hard (<=100 TB)', '>100 TB']
inscope_vinfo_df['Disk Size Tiers'] = pd.cut(inscope_vinfo_df['Disk Size TB'], bins=disk_size_bins, labels=disk_bin_labels)

# Create a pivot table based on disk size tiers
disk_tier_pivot_df = inscope_vinfo_df.pivot_table(
    index='Disk Size Tiers', 
    values=['VM', 'Disk Size TB'], 
    aggfunc={'VM': 'count', 'Disk Size TB': 'sum'}, 
    observed=False  # Prevent future warning and maintain current behavior
)

# Add total row with rounded 'Disk Size TB' to nearest 0.5 TB
total_row = pd.DataFrame({
    'VM': [disk_tier_pivot_df['VM'].sum()],
    'Disk Size TB': [round(disk_tier_pivot_df['Disk Size TB'].sum() * 2) / 2]  # Rounding to nearest 0.5 TB
}, index=['Total'])

# Combine the pivot table with the total row
disk_tier_pivot_with_total = pd.concat([disk_tier_pivot_df, total_row])

# Format the table for display
formatted_table = disk_tier_pivot_with_total.copy()
formatted_table['Disk Size TB'] = formatted_table['Disk Size TB'].apply(lambda x: f"{x:,.2f}")
formatted_table['VM'] = formatted_table['VM'].apply(lambda x: f"{int(x):,}")

# Display the table
print("Disk Tier Summary with Total (Formatted):")
print(formatted_table.to_string())

# Calculate percentages for VM distribution in each disk tier
total_vms = disk_tier_pivot_df['VM'].sum()
percentages = (disk_tier_pivot_df['VM'] / total_vms) * 100

# Generate labels for the legend
labels = [f"{tier} - {pct:.1f}% ({count} VMs)" for tier, pct, count in zip(disk_tier_pivot_df.index, percentages, disk_tier_pivot_df['VM'])]

# Define colors for the pie chart (green, yellow, red scheme)
tier_colors = ['green', 'yellow', 'orange', 'red', 'darkred', 'gray']

# Pie Chart 1: Standard Pie Chart with Legend
plt.figure(figsize=(8, 8))
plt.pie(disk_tier_pivot_df['VM'], startangle=140, colors=plt.cm.Paired.colors, wedgeprops={'edgecolor': 'black', 'linewidth': 1})
plt.title('VM Distribution by Disk Tier for In-Scope VMs (With Legend)', fontsize=12)
plt.legend(labels, loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
plt.tight_layout()
plt.show()

# Pie Chart 2: Pie Chart with Green, Yellow, Red Scheme (No Labels on Slices)
plt.figure(figsize=(8, 8))
plt.pie(disk_tier_pivot_df['VM'], colors=tier_colors, autopct='%1.1f%%', startangle=140, wedgeprops={'edgecolor': 'black', 'linewidth': 1})
plt.title('VM Distribution by Tier (Easy, Medium, Hard, etc...)', fontsize=12)
plt.legend(labels, loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
plt.tight_layout()
plt.show()

### 6- Categorize Host Compute Nodes by vCenter

Categorize all the compute nodes by the model number for each vCenter separately.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Group all the hosts by their model and vCenter
all_host_model_vcenter_pivot_df = consolidated_vhost_df.pivot_table(
    index=['vCenter', 'Vendor', 'Model'],
    values=['Host'],
    aggfunc={'Host': 'count'},
    observed=False,
    margins=False,
    sort=True
)

print(f'Distribution of ALL host models by vCenter:\n')
print(all_host_model_vcenter_pivot_df)
all_host_model_vcenter_pivot_df.to_clipboard(excel=True)

# Bar chart creation based on the pivot table (sorted by host count)
plt.figure(figsize=(12, 8))  # Increase figure size for better visibility

# Prepare labels and data
labels = all_host_model_vcenter_pivot_df.index.map(lambda x: f'{x[1]} {x[2]} ({x[0]})')  # Combine Vendor, Model, and vCenter in the label
sizes = all_host_model_vcenter_pivot_df['Host']

# Choose a color palette
colors = plt.cm.tab20(np.linspace(0, 1, len(sizes)))

# Create the bar chart
plt.barh(labels, sizes, color=colors, edgecolor='black')  # Horizontal bar chart

# Add title and formatting
plt.title('In-Scope Host Node Compute Models', fontsize=12)
plt.xlabel('Number of Hosts', fontsize=10)
plt.ylabel('Host Model', fontsize=10)

# Display the bar chart
plt.tight_layout()  # Adjust layout to prevent clipping
plt.show()

### 7- Count the ESXi Clusters with In-Scope VMs

Count the total ESXi clusters

In [ ]:
# Get unique Datacenters
datacenters = filtered_vinfo_df['Datacenter'].unique()

# Print total number of in-scope VMware Datacenters
print(f'Total in-scope VMware Datacenters: {len(datacenters):,}')

# Get unique Clusters
clusters = filtered_vinfo_df['Cluster'].unique()

# Print total number of in-scope ESXi clusters
print(f'Total in-scope ESXi clusters: {len(clusters):,}')

### 8- VM distribution by ESXi Clusters

This is orthogonal to the VM distribution analysis by `vCenters`, as a vCenter is likely to contain multiple `clusters.

In [ ]:
# Pivot the VM information on the clusters
inscope_cluster_pivot = inscope_vinfo_df.pivot_table(index='Cluster',
                                                      values=['VM'],
                                                      aggfunc={'VM': 'count'},
                                                      sort=True)

# Calculate the total number of VMs
total_vms = inscope_cluster_pivot['VM'].sum()

# Calculate the total number of clusters
total_clusters = inscope_cluster_pivot.index.nunique()

print(f'Distribution In-Scope VMs to Clusters:\n')
print(inscope_cluster_pivot)
print(f'\nTotal VMs across all clusters: {total_vms}')
print(f'Total number of clusters: {total_clusters}')
inscope_cluster_pivot.to_clipboard(excel=True)

# Create a pie chart
plt.figure(figsize=(8, 8))
plt.pie(inscope_cluster_pivot['VM'], labels=inscope_cluster_pivot.index, autopct='%1.1f%%', startangle=140)
plt.title('VMs distributed to clusters')
plt.show()

### 9- VMs Categorized by Environment

This analysis is separate from the VM distribution by environment. The goal is to enhance the vInfo data with additional categorization.

1. Create an Environment Column: This column will be based on the name of the ESXi cluster, which indicates the site location.
2. Add a Site-Type Column: This column will categorize the environment into one of the following types: Prod, QA, Test, or NonProd.
3. Handle Unclassified Clusters: Any cluster names that do not fall into the four categories above will be grouped as "Unknown."

In [ ]:
##### VMs Categorized by Environment
import pandas as pd
import matplotlib.pyplot as plt

# Define a function to categorize 'Cluster' into environments based on 'Environment' column or 'Cluster' name
def determine_environment(row):
    # Ensure 'Environment' column is checked first
    environment = str(row.get('Environment', '')).strip().lower()
    if environment in ['nonprod', 'non-production', 'nonproduction']:
        return 'NonProd'
    elif environment in ['prod', 'production']:
        return 'Prod'
    elif environment in ['dev', 'development']:
        return 'Dev'
    elif environment in ['qa']:
        return 'QA'
    elif environment in ['test']:
        return 'Test'

    # If 'Environment' is empty, check 'Cluster' name for environment keywords
    cluster_name = str(row.get('Cluster', '')).strip().lower()
    if 'nonprod' in cluster_name:
        return 'NonProd'
    elif 'prod' in cluster_name:
        return 'Prod'
    elif 'dev' in cluster_name:
        return 'Dev'
    elif 'qa' in cluster_name:
        return 'QA'
    elif 'test' in cluster_name:
        return 'Test'
    
    # Return 'Unknown' if no matching environment is found
    return 'Unknown'

# Apply the function to create or update the 'Environment' column in the DataFrame
inscope_vinfo_df['Environment'] = inscope_vinfo_df.apply(determine_environment, axis=1)

# Display the distribution of environments
print(f"Environment Distribution:\n")
environment_counts = inscope_vinfo_df['Environment'].value_counts()
for env, count in environment_counts.items():
    print(f"{env}: {count}")

### 10- VMs Graphed by Environment

In [ ]:
##### Block Display by Environment

# Count the VMs classified by each environment type
print(f"Total In-scope VMs in Prod   : {len(inscope_vinfo_df[inscope_vinfo_df['Environment'] == 'Prod']):,}")
print(f"Total In-scope VMs in Dev    : {len(inscope_vinfo_df[inscope_vinfo_df['Environment'] == 'Dev']):,}")
print(f"Total In-scope VMs in QA     : {len(inscope_vinfo_df[inscope_vinfo_df['Environment'] == 'QA']):,}")
print(f"Total In-scope VMs in Test   : {len(inscope_vinfo_df[inscope_vinfo_df['Environment'] == 'Test']):,}")
print(f"Total In-scope VMs in NonProd : {len(inscope_vinfo_df[inscope_vinfo_df['Environment'] == 'NonProd']):,}")
print(f"Total In-scope VMs in Unknown: {len(inscope_vinfo_df[inscope_vinfo_df['Environment'] == 'Unknown']):,}")

import matplotlib.pyplot as plt

# Count the number of VMs in each environment type
env_counts = inscope_vinfo_df['Environment'].value_counts()

# Plot the pie chart
plt.figure(figsize=(8, 8))
plt.pie(env_counts, labels=env_counts.index, autopct='%1.1f%%', startangle=140, colors=plt.cm.tab20.colors, wedgeprops={'edgecolor': 'black'})
plt.title('VM Distribution by Environment')

# Display the pie chart
plt.axis('equal')  # Equal aspect ratio ensures that pie chart is drawn as a circle.
plt.show()

### 10- Summarize Operating Systems by Supported vs. Unsupported

In this section, we provide a summary of the supported and unsupported operating systems for the in-scope vCenter instances.

In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display

# Define the list of supported OSs for categorization
supported_os_list = [
    "Red Hat Enterprise Linux 7 (64-bit)", "Red Hat Enterprise Linux 8 (64-bit)", "Red Hat Enterprise Linux 9 (64-bit)",
    "Microsoft Windows 10 (64-bit)", "Microsoft Windows 11 (64-bit)", "Microsoft Windows Server 2012 (64-bit)",
    "Microsoft Windows Server 2016 (64-bit)", "Microsoft Windows Server 2016 or later (64-bit)", 
    "Microsoft Windows Server 2019 (64-bit)", "Microsoft Windows Server 2022 (64-bit)", 
    "SUSE Linux Enterprise 15 (64-bit)", "SUSE Linux Enterprise 12 (64-bit)", 
    "Ubuntu 18.04 (64-bit)", "Ubuntu 20.04 (64-bit)", "Ubuntu 22.04 (64-bit)", 
    "Ubuntu 24.04 (64-bit)", "Fedora 30 (64-bit)", "Fedora 40 (64-bit)", 
    "CentOS 9 (64-bit)", "CentOS 8 (64-bit)"
]

# Combine the OS columns into one "Final OS"
inscope_vinfo_df['Final OS'] = inscope_vinfo_df['OS according to the VMware Tools'].combine_first(inscope_vinfo_df['OS according to the configuration file'])

# Separate supported and unsupported OS based on "Final OS"
supported_final_df = inscope_vinfo_df[inscope_vinfo_df['Final OS'].isin(supported_os_list)]
unsupported_final_df = inscope_vinfo_df[~inscope_vinfo_df['Final OS'].isin(supported_os_list)]

# Create pivot tables for supported and unsupported OS
supported_final_pivot = supported_final_df.pivot_table(index='Final OS', values='VM', aggfunc='count')
unsupported_final_pivot = unsupported_final_df.pivot_table(index='Final OS', values='VM', aggfunc='count')

# Add totals to the pivot tables
supported_final_total = supported_final_pivot['VM'].sum()
unsupported_final_total = unsupported_final_pivot['VM'].sum()
supported_final_pivot.loc['Total'] = supported_final_total
unsupported_final_pivot.loc['Total'] = unsupported_final_total

# Display the pivot tables
print("Supported Operating Systems (Final OS) with VM Counts:")
display(supported_final_pivot)
print("\nUnsupported Operating Systems (Final OS) with VM Counts:")
display(unsupported_final_pivot)

# Plotting supported and unsupported OS bar charts
plt.figure(figsize=(12, 6))
plt.bar(supported_final_pivot.index, supported_final_pivot['VM'], color='green')
plt.xlabel('Supported Operating Systems (Final OS)')
plt.ylabel('Number of VMs')
plt.title('Supported Operating Systems with VM Counts (Final OS)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 6))
plt.bar(unsupported_final_pivot.index, unsupported_final_pivot['VM'], color='red')
plt.xlabel('Unsupported Operating Systems (Final OS)')
plt.ylabel('Number of VMs')
plt.title('Unsupported Operating Systems with VM Counts (Final OS)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Process VMware Tools and Configuration File OS columns similarly
def process_os_column(os_column_name):
    supported_df = inscope_vinfo_df[inscope_vinfo_df[os_column_name].isin(supported_os_list)]
    unsupported_df = inscope_vinfo_df[~inscope_vinfo_df[os_column_name].isin(supported_os_list)]
    
    supported_pivot = supported_df.pivot_table(index=os_column_name, values='VM', aggfunc='count')
    unsupported_pivot = unsupported_df.pivot_table(index=os_column_name, values='VM', aggfunc='count')
    
    # Add totals
    supported_total = supported_pivot['VM'].sum()
    unsupported_total = unsupported_pivot['VM'].sum()
    supported_pivot.loc['Total'] = supported_total
    unsupported_pivot.loc['Total'] = unsupported_total
    
    return supported_pivot, unsupported_pivot

# Process for "OS according to VMware Tools"
supported_vmware_tools_pivot, unsupported_vmware_tools_pivot = process_os_column('OS according to the VMware Tools')

# Display and plot VMware Tools OS counts
print("\nSupported Operating Systems (VMware Tools) with VM Counts:")
display(supported_vmware_tools_pivot)
print("\nUnsupported Operating Systems (VMware Tools) with VM Counts:")
display(unsupported_vmware_tools_pivot)

# Plot VMware Tools bar charts
plt.figure(figsize=(12, 6))
plt.bar(supported_vmware_tools_pivot.index, supported_vmware_tools_pivot['VM'], color='green')
plt.xlabel('Supported Operating Systems (VMware Tools)')
plt.ylabel('Number of VMs')
plt.title('Supported Operating Systems with VM Counts (VMware Tools)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 6))
plt.bar(unsupported_vmware_tools_pivot.index, unsupported_vmware_tools_pivot['VM'], color='red')
plt.xlabel('Unsupported Operating Systems (VMware Tools)')
plt.ylabel('Number of VMs')
plt.title('Unsupported Operating Systems with VM Counts (VMware Tools)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Process for "OS according to the Configuration File"
supported_config_file_pivot, unsupported_config_file_pivot = process_os_column('OS according to the configuration file')

# Display and plot Configuration File OS counts
print("\nSupported Operating Systems (Configuration File) with VM Counts:")
display(supported_config_file_pivot)
print("\nUnsupported Operating Systems (Configuration File) with VM Counts:")
display(unsupported_config_file_pivot)

# Plot Configuration File bar charts
plt.figure(figsize=(12, 6))
plt.bar(supported_config_file_pivot.index, supported_config_file_pivot['VM'], color='green')
plt.xlabel('Supported Operating Systems (Configuration File)')
plt.ylabel('Number of VMs')
plt.title('Supported Operating Systems with VM Counts (Configuration File)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 6))
plt.bar(unsupported_config_file_pivot.index, unsupported_config_file_pivot['VM'], color='red')
plt.xlabel('Unsupported Operating Systems (Configuration File)')
plt.ylabel('Number of VMs')
plt.title('Unsupported Operating Systems with VM Counts (Configuration File)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

### 11- Migration Complexity In-Scope VMs by OS

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import re

# Define the list of supported OSs and specific non-supported OSes
supported_os_list = [
    "Red Hat Enterprise Linux 7 (64-bit)", "Red Hat Enterprise Linux 8 (64-bit)", "Red Hat Enterprise Linux 9 (64-bit)",
    "Microsoft Windows 10 (64-bit)", "Microsoft Windows 11 (64-bit)", 
    "Microsoft Windows Server 2016 (64-bit)", "Microsoft Windows Server 2019 (64-bit)", 
    "Microsoft Windows Server 2022 (64-bit)", "Microsoft Windows Server 2016 or later (64-bit)",
    "SUSE Linux Enterprise 15 (64-bit)", "SUSE Linux Enterprise 12 (64-bit)", 
    "Ubuntu 18.04 (64-bit)", "Ubuntu 20.04 (64-bit)", "Ubuntu 22.04 (64-bit)", 
    "Ubuntu 24.04 (64-bit)", "Fedora 30 (64-bit)", "Fedora 40 (64-bit)", 
    "CentOS 9 (64-bit)", "CentOS 8 (64-bit)"
]

easy_os_list = [
    "Oracle Linux 7 (64-bit)", "Oracle Linux 8 (64-bit)", "Oracle Linux 9 (64-bit)",
    "CentOS 7 (64-bit)", "Red Hat Enterprise Linux 6 (64-bit)"
]

# Create dictionaries for exact lookups
supported_os_dict = {os: True for os in supported_os_list}
easy_os_dict = {os: True for os in easy_os_list}

# Pre-process the Final OS column
inscope_vinfo_df['Final OS'] = inscope_vinfo_df['Final OS'].fillna("Unknown OS").astype(str).str.strip()

# Preprocess the Cluster column
inscope_vinfo_df['Cluster'] = inscope_vinfo_df['Cluster'].fillna('').str.lower()

# Function to classify Database Group (SQL/Oracle/Other)
def classify_database_group(cluster_name):
    if 'sql-' in cluster_name:
        return 'SQL'
    elif 'ora' in cluster_name:
        return 'Oracle'
    return 'Other'

inscope_vinfo_df['Database Group'] = inscope_vinfo_df['Cluster'].apply(classify_database_group)

# Classification function
def classify_disk_size(disk_size_tb, os_name, network, database_group):
    os_name_stripped = os_name.strip()
    
    # Database-specific groupings
    if database_group == 'SQL':
        base_group = "SQL-DBs - "
    elif database_group == 'Oracle':
        base_group = "Oracle-DBs - "
    else:
        base_group = ""
    
    # Check if Network contains F5
    if 'F5' in str(network).upper():
        return f"{base_group}F5-Hard"
    
    # Easy criteria
    if disk_size_tb <= 2 and (easy_os_dict.get(os_name_stripped) or (supported_os_dict.get(os_name_stripped) and not os_name_stripped.startswith("Microsoft"))):
        if easy_os_dict.get(os_name_stripped):
            return f"{base_group}Easy - Unsupported"
        return f"{base_group}Easy"
    
    # Medium criteria
    if (2 < disk_size_tb <= 10 and (supported_os_dict.get(os_name_stripped) or easy_os_dict.get(os_name_stripped))) or \
       (disk_size_tb <= 2 and os_name_stripped.startswith("Microsoft") and supported_os_dict.get(os_name_stripped)):
        if easy_os_dict.get(os_name_stripped):
            return f"{base_group}Medium - Unsupported"
        return f"{base_group}Medium"
    
    # Hard criteria
    if disk_size_tb > 10 and (supported_os_dict.get(os_name_stripped) or easy_os_dict.get(os_name_stripped)):
        if easy_os_dict.get(os_name_stripped):
            return f"{base_group}Hard - Unsupported"
        return f"{base_group}Hard"
    
    # Hard - OS Unsupported
    return f"{base_group}Hard - OS Unsupported"

# Apply the classification logic
inscope_vinfo_df['Disk Classification'] = inscope_vinfo_df.apply(
    lambda row: classify_disk_size(row['Disk Size TB'], row['Final OS'], row['Network #1'], row['Database Group']),
    axis=1
)

# Combine unsupported groups into "Unsupported OS"
def combine_unsupported_classifications(classification):
    if "Unsupported" in classification:
        if "Hard" in classification:
            return "Hard - Unsupported OS"
        elif "Medium" in classification:
            return "Medium - Unsupported OS"
        elif "Easy" in classification:
            return "Easy - Unsupported OS"
    return classification

inscope_vinfo_df['Combined Classification'] = inscope_vinfo_df['Disk Classification'].apply(combine_unsupported_classifications)

# Summarize counts by combined classification
disk_classification_summary = inscope_vinfo_df.pivot_table(
    index='Combined Classification',
    values='VM',
    aggfunc='count',
    fill_value=0
)
disk_classification_summary = disk_classification_summary[disk_classification_summary['VM'] > 0]

# Assign colors based on classification
classification_colors = {
    'Easy': 'green',
    'Medium': 'yellow',
    'Hard': 'red',
    'Unsupported OS': 'gray'
}

def get_color_for_classification(classification):
    if "Easy" in classification:
        return classification_colors['Easy']
    elif "Medium" in classification:
        return classification_colors['Medium']
    elif "Hard" in classification:
        return classification_colors['Hard']
    return classification_colors.get("Unsupported OS", 'gray')  # Default to gray for unsupported OS

colors = [get_color_for_classification(cls) for cls in disk_classification_summary.index]

# Plot the bar chart with color coding
plt.figure(figsize=(12, 6))
plt.bar(disk_classification_summary.index, disk_classification_summary['VM'], color=colors)
plt.xlabel('Classification')
plt.ylabel('Number of VMs')
plt.title('VM Distribution by Classification')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Extract unsupported OS breakdown
unsupported_combined_df = inscope_vinfo_df[
    inscope_vinfo_df['Disk Classification'].str.contains("Unsupported", na=False)
].drop_duplicates(subset=['VM'])

unsupported_os_summary = unsupported_combined_df.groupby('Final OS').size().reset_index(name='VM Count')
unsupported_os_summary = unsupported_os_summary[unsupported_os_summary['VM Count'] > 0]

# Print unsupported OS breakdown
if not unsupported_os_summary.empty:
    print("\nUnsupported OS Breakdown:")
    print(unsupported_os_summary)

### 12- Migration Complexity In-Scope VMs by Disk

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assume 'inscope_vinfo_df' contains all required data, including 'Environment' and 'Disk Size TB'

# Define disk size categories
disk_bins = [0, 2, float('inf')]
disk_labels = ['<= 2TB', '> 2TB']

# Create a column to categorize disk size into <= 2TB and > 2TB
inscope_vinfo_df['Disk Size Category'] = pd.cut(inscope_vinfo_df['Disk Size TB'], bins=disk_bins, labels=disk_labels)

# Create a pivot table to get VM totals for each environment and disk size category
env_disk_summary = inscope_vinfo_df.pivot_table(
    index='Environment',
    columns='Disk Size Category',
    values='VM',
    aggfunc='count',
    fill_value=0,
    observed=True  # Explicitly specify observed=True
).reset_index()

# Dynamically rename columns based on the actual columns in the pivot table
env_disk_summary.columns = ['Environment'] + [col if isinstance(col, str) else col[1] for col in env_disk_summary.columns[1:]]

# Display the summary table for each environment with VM totals by disk size category
print("VM Totals by Environment and Disk Size Category:")
print(env_disk_summary.to_string(index=False, header=True))

# Plotting (optional)

# Bar chart for VM Disk Size Totals by Environment
env_disk_summary.plot(
    x='Environment', 
    kind='bar', 
    stacked=False, 
    figsize=(10, 6), 
    width=0.8
)
plt.xlabel('Environment')
plt.ylabel('Number of VMs')
plt.title('VM Totals by Environment and Disk Size Category')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

### 13- Estimated Migration Time by OS

In [ ]:
import pandas as pd
import math

# Constants
migration_time_per_500gb = 110  # minutes (1 hour 50 minutes per 500GB)
fte_hours_per_day = 8           # 8 hours per day per FTE
fte_count = 10                  # 10 FTEs available
pmt_hours = 1.5                 # Post-migration troubleshooting time per VM in hours

# Example list of supported operating systems (uncomment and modify as needed)
supported_os_list = ['Windows Server', 'Red Hat', 'Oracle Linux']  # Add your OS list here

# Ensure required columns are present in the existing inscope_vinfo_df
required_columns = ['Environment', 'Final OS', 'Disk Size TB', 'VM', 'Cluster', 'Disk Classification']
missing_columns = [col for col in required_columns if col not in inscope_vinfo_df.columns]

if missing_columns:
    raise ValueError(f"The following required columns are missing in inscope_vinfo_df: {missing_columns}")

# Preprocess columns
inscope_vinfo_df['Final OS'] = inscope_vinfo_df['Final OS'].fillna("Unknown OS").astype(str).str.strip()
inscope_vinfo_df['Environment'] = inscope_vinfo_df['Environment'].fillna('unknown').str.lower().str.strip()
inscope_vinfo_df['Disk Size TB'] = pd.to_numeric(inscope_vinfo_df['Disk Size TB'], errors='coerce')
inscope_vinfo_df['VM'] = inscope_vinfo_df['VM'].fillna('Unknown VM')
inscope_vinfo_df['Cluster'] = inscope_vinfo_df['Cluster'].fillna('').str.lower()

# Derive F5-Hard complexity directly from Disk Classification
inscope_vinfo_df['Complexity'] = inscope_vinfo_df['Disk Classification'].apply(
    lambda x: 'F5-Hard' if 'F5-Hard' in str(x) else None
)

# Assign other complexities only for non-F5-Hard rows
def classify_complexity(row):
    if row['Complexity'] == 'F5-Hard':  # Skip if already classified as F5-Hard
        return 'F5-Hard'
    if 'sql-' in row['Cluster']:
        return 'MSSQL-DBs'  # Changed from SQL-DBs to MSSQL-DBs
    elif 'ora' in row['Cluster']:
        return 'Oracle-DBs'
    elif row['Disk Size TB'] <= 2:
        return 'Easy'
    elif row['Disk Size TB'] <= 5:
        return 'Medium'
    elif row['Disk Size TB'] <= 10:
        return 'Hard'
    else:
        return 'Hard - Very Large'

inscope_vinfo_df['Complexity'] = inscope_vinfo_df.apply(classify_complexity, axis=1)

# Define the custom sorting order for Complexity
complexity_order = ['Easy', 'Medium', 'Hard', 'F5-Hard', 'Hard - Very Large', 'Oracle-DBs', 'MSSQL-DBs']
inscope_vinfo_df['Complexity'] = pd.Categorical(
    inscope_vinfo_df['Complexity'],
    categories=complexity_order,
    ordered=True
)

# Sort the DataFrame by Complexity
inscope_vinfo_df = inscope_vinfo_df.sort_values('Complexity')

# Create the OS Support column based on supported_os_list
inscope_vinfo_df['OS Support'] = inscope_vinfo_df['Final OS'].apply(
    lambda os: 'Supported' if any(supported_os in os for supported_os in supported_os_list) else 'Not Supported'
)

# Add Migration Time
inscope_vinfo_df['Migration Time (minutes)'] = inscope_vinfo_df['Disk Size TB'].apply(
    lambda size: ((size * 1024) / 500) * migration_time_per_500gb
)

# Add Post-Migration Troubleshooting Time
pmt_minutes = pmt_hours * 60
inscope_vinfo_df['Total Time (minutes)'] = inscope_vinfo_df['Migration Time (minutes)'] + pmt_minutes

# Summarize by Complexity and OS Support
disk_classification_summary = inscope_vinfo_df.groupby(['Complexity', 'OS Support'], observed=True).agg(
    VM_Count=('VM', 'count'),
    Total_Disk=('Disk Size TB', 'sum'),
    Total_Mig_Time_Minutes=('Total Time (minutes)', 'sum')
).reset_index()

# Include all "OS Support" categories in the summary
for complexity in complexity_order:
    for os_support in ['Supported', 'Not Supported']:
        if not ((disk_classification_summary['Complexity'] == complexity) &
                (disk_classification_summary['OS Support'] == os_support)).any():
            disk_classification_summary = pd.concat([
                disk_classification_summary,
                pd.DataFrame({
                    'Complexity': [complexity],
                    'OS Support': [os_support],
                    'VM_Count': [0],
                    'Total_Disk': [0],
                    'Total_Mig_Time_Minutes': [0]
                })
            ], ignore_index=True)

# Filter out rows where VM_Count is 0
disk_classification_summary = disk_classification_summary[disk_classification_summary['VM_Count'] > 0]

# Formatting for Migration Time and Days Calculation (1 decimal place)
disk_classification_summary['Formatted_Mig_Time'] = disk_classification_summary['Total_Mig_Time_Minutes'].apply(
    lambda minutes: f"{minutes / 60:,.1f}h"  # Converted minutes to hours with 1 decimal place
)
disk_classification_summary['Days_Per_FTEs'] = disk_classification_summary['Total_Mig_Time_Minutes'].apply(
    lambda minutes: f"{minutes / (fte_hours_per_day * 60 * fte_count):,.1f}"  # 1 decimal place
)

# Format VM Count with thousands separator
disk_classification_summary['VM_Count'] = disk_classification_summary['VM_Count'].apply(lambda x: f"{x:,}")

# Format Total Disk (TB) without rounding
disk_classification_summary['Total_Disk'] = disk_classification_summary['Total_Disk'].apply(lambda x: f"{x:,.2f}")

# Add totals row
totals_row = {
    'Complexity': 'Totals',
    'OS Support': '',
    'VM_Count': f"{int(disk_classification_summary['VM_Count'].str.replace(',', '').astype(int).sum()):,}",
    'Total_Disk': f"{float(disk_classification_summary['Total_Disk'].str.replace(',', '').astype(float).sum()):,.2f}",
    'Formatted_Mig_Time': f"{disk_classification_summary['Total_Mig_Time_Minutes'].sum() / 60:,.1f}h",
    'Days_Per_FTEs': f"{disk_classification_summary['Total_Mig_Time_Minutes'].sum() / (fte_hours_per_day * 60 * fte_count):,.1f}"
}
disk_classification_summary = pd.concat(
    [disk_classification_summary, pd.DataFrame([totals_row])],
    ignore_index=True
)

# Print the table
headers = [
    "Complexity", "OS Support", "VM Count", "Total Disk (TB)",
    "Total Migration Time", f"Days ({fte_count} FTEs)"
]
rows = disk_classification_summary[
    ['Complexity', 'OS Support', 'VM_Count', 'Total_Disk', 'Formatted_Mig_Time', 'Days_Per_FTEs']
].values.tolist()

def custom_table_format_with_totals(headers, rows):
    horizontal_line = "─"
    vertical_line = "│"
    corner_tl, corner_tr = "╭", "╮"
    corner_bl, corner_br = "╰", "╯"
    join_t, join_b, join_c = "┬", "┴", "┼"
    col_widths = [max(len(str(item)) for item in col) for col in zip(headers, *rows)]
    def make_row(items):
        return vertical_line + vertical_line.join(f"{str(item).rjust(width)}" for item, width in zip(items, col_widths)) + vertical_line
    top_line = corner_tl + join_t.join(horizontal_line * width for width in col_widths) + corner_tr
    header_row = make_row(headers)
    divider_row = join_c.join(horizontal_line * width for width in col_widths).join(["├", "┤"])
    data_rows = [make_row(row) for row in rows[:-1]]
    totals_divider_row = join_c.join(horizontal_line * width for width in col_widths).join(["├", "┤"])
    totals_row = make_row(rows[-1])
    bottom_line = corner_bl + join_b.join(horizontal_line * width for width in col_widths) + corner_br
    return "\n".join([top_line, header_row, divider_row] + data_rows + [totals_divider_row, totals_row, bottom_line])

print("\nSummary Table:")
print(custom_table_format_with_totals(headers, rows))

### 14- Estimated Migration Time by vCenter

In [ ]:
import pandas as pd

# Constants
fte_hours_per_day = 8           # 8 hours per day per FTE
fte_count = 10                  # 10 FTEs available
complexity_order = ['Easy', 'Medium', 'Hard', 'F5-Hard', 'Hard - Very Large', 'Oracle-DBs', 'MSSQL-DBs']

# Assuming 'inscope_vinfo_df' is the DataFrame that already exists
vcenters = inscope_vinfo_df['vCenter'].unique()

# Create a dictionary to store summaries for each vCenter
vcenter_summaries = {}

# Function to format the table output
def custom_table_format_with_totals(headers, rows):
    horizontal_line = "─"
    vertical_line = "│"
    corner_tl, corner_tr = "╭", "╮"
    corner_bl, corner_br = "╰", "╯"
    join_t, join_b, join_c = "┬", "┴", "┼"
    col_widths = [max(len(str(item)) for item in col) for col in zip(headers, *rows)]
    
    def make_row(items):
        return vertical_line + vertical_line.join(f"{str(item).rjust(width)}" for item, width in zip(items, col_widths)) + vertical_line
    
    top_line = corner_tl + join_t.join(horizontal_line * width for width in col_widths) + corner_tr
    header_row = make_row(headers)
    divider_row = join_c.join(horizontal_line * width for width in col_widths).join(["├", "┤"])
    data_rows = [make_row(row) for row in rows[:-1]]
    totals_divider_row = join_c.join(horizontal_line * width for width in col_widths).join(["├", "┤"])
    totals_row = make_row(rows[-1])
    bottom_line = corner_bl + join_b.join(horizontal_line * width for width in col_widths) + corner_br
    return "\n".join([top_line, header_row, divider_row] + data_rows + [totals_divider_row, totals_row, bottom_line])

# Process each vCenter separately
for vcenter in vcenters:
    # Filter the DataFrame for the current vCenter
    vcenter_df = inscope_vinfo_df[inscope_vinfo_df['vCenter'] == vcenter]
    
    # Perform the same calculations as before on the filtered DataFrame
    disk_classification_summary = vcenter_df.groupby(['Complexity', 'OS Support'], observed=True).agg(
        VM_Count=('VM', 'count'),
        Total_Disk=('Disk Size TB', 'sum'),
        Total_Mig_Time_Minutes=('Total Time (minutes)', 'sum')
    ).reset_index()

    # Include all "OS Support" categories in the summary
    for complexity in complexity_order:
        for os_support in ['Supported', 'Not Supported']:
            if not ((disk_classification_summary['Complexity'] == complexity) &
                    (disk_classification_summary['OS Support'] == os_support)).any():
                disk_classification_summary = pd.concat([
                    disk_classification_summary,
                    pd.DataFrame({
                        'Complexity': [complexity],
                        'OS Support': [os_support],
                        'VM_Count': [0],
                        'Total_Disk': [0],
                        'Total_Mig_Time_Minutes': [0]
                    })
                ], ignore_index=True)

    # Filter out rows where VM_Count is 0
    disk_classification_summary = disk_classification_summary[disk_classification_summary['VM_Count'] > 0]
    
    # Convert 'Complexity' to a categorical type with the custom order
    disk_classification_summary['Complexity'] = pd.Categorical(
        disk_classification_summary['Complexity'],
        categories=complexity_order,
        ordered=True
    )

    # Sort the summary by the custom complexity order
    disk_classification_summary = disk_classification_summary.sort_values('Complexity')

    # Add calculated columns for formatted migration time and days (1 decimal place)
    disk_classification_summary['Formatted_Mig_Time'] = disk_classification_summary['Total_Mig_Time_Minutes'].apply(
        lambda minutes: f"{minutes / 60:,.1f}h"  # Exact value in hours with 1 decimal place
    )
    disk_classification_summary['Days_Per_FTEs'] = disk_classification_summary['Total_Mig_Time_Minutes'].apply(
        lambda minutes: f"{minutes / (fte_hours_per_day * 60 * fte_count):,.1f}"  # Exact value in days with 1 decimal place
    )

    # Format VM Count with thousands separator
    disk_classification_summary['VM_Count'] = disk_classification_summary['VM_Count'].apply(lambda x: f"{x:,}")
    disk_classification_summary['Total_Disk'] = disk_classification_summary['Total_Disk'].apply(lambda x: f"{x:,.0f}")

    # Add totals row
    totals_row = {
        'Complexity': 'Totals',
        'OS Support': '',
        'VM_Count': f"{int(disk_classification_summary['VM_Count'].str.replace(',', '').astype(int).sum()):,}",
        'Total_Disk': f"{float(disk_classification_summary['Total_Disk'].str.replace(',', '').astype(float).sum()):,.0f}",
        'Formatted_Mig_Time': f"{disk_classification_summary['Total_Mig_Time_Minutes'].sum() / 60:,.1f}h",
        'Days_Per_FTEs': f"{disk_classification_summary['Total_Mig_Time_Minutes'].sum() / (fte_hours_per_day * 60 * fte_count):,.1f}"
    }
    disk_classification_summary = pd.concat(
        [disk_classification_summary, pd.DataFrame([totals_row])],
        ignore_index=True
    )

    # Store the summary in the dictionary
    vcenter_summaries[vcenter] = disk_classification_summary

    # Print the table for the current vCenter
    print(f"\nSummary Table for vCenter: {vcenter}")
    headers = [
        "Complexity", "OS Support", "VM Count", "Total Disk (TB)",
        "Total Migration Time", f"Days ({fte_count} FTEs)"
    ]
    rows = disk_classification_summary[
        ['Complexity', 'OS Support', 'VM_Count', 'Total_Disk', 'Formatted_Mig_Time', 'Days_Per_FTEs']
    ].values.tolist()
    print(custom_table_format_with_totals(headers, rows))

# Calculate global totals across all vCenters
global_total_mig_time = sum(
    summary['Total_Mig_Time_Minutes'].sum() for summary in vcenter_summaries.values()
)
global_total_days = global_total_mig_time / (fte_hours_per_day * 60 * fte_count)

# Calculate aggregate FTE days from vCenter summaries
aggregate_total_days = sum(
    summary.loc[summary['Complexity'] == 'Totals', 'Days_Per_FTEs'].astype(float).sum() for summary in vcenter_summaries.values()
)

# Display Global Totals (1 decimal place)
print(f"\nGlobal Total Migration Time: {global_total_mig_time / 60:,.1f}h")
print(f"Global Total FTE Days (10 FTEs): {global_total_days:,.1f}")
print(f"Aggregate FTE Days (10 FTEs): {aggregate_total_days:,.1f}")

### 15- Estimated Migration Time (Summary)

In [ ]:
#info
import pandas as pd
import math
import matplotlib.pyplot as plt

# Constants
migration_time_per_500gb = 110  # minutes (1 hour 50 minutes per 500GB)
fte_hours_per_day = 8           # 8 hours per day per FTE
fte_count = 10                  # 10 FTEs available
pmt_hours = 1.5                 # Post-migration troubleshooting time per VM in hours

# Example list of supported operating systems
supported_os_list = ['Windows Server', 'Red Hat', 'Oracle Linux']

# Ensure required columns are present in the existing inscope_vinfo_df
required_columns = ['Environment', 'Final OS', 'Disk Size TB', 'VM', 'Cluster', 'Disk Classification']
missing_columns = [col for col in required_columns if col not in inscope_vinfo_df.columns]

if missing_columns:
    raise ValueError(f"The following required columns are missing in inscope_vinfo_df: {missing_columns}")

# Preprocess columns
inscope_vinfo_df['Final OS'] = inscope_vinfo_df['Final OS'].fillna("Unknown OS").astype(str).str.strip()
inscope_vinfo_df['Environment'] = inscope_vinfo_df['Environment'].fillna('unknown').str.lower().str.strip()
inscope_vinfo_df['Disk Size TB'] = pd.to_numeric(inscope_vinfo_df['Disk Size TB'], errors='coerce')
inscope_vinfo_df['VM'] = inscope_vinfo_df['VM'].fillna('Unknown VM')
inscope_vinfo_df['Cluster'] = inscope_vinfo_df['Cluster'].fillna('').str.lower()

# Debug: Check unique OS values
print("Unique Final OS values:", inscope_vinfo_df['Final OS'].unique())

# Derive F5-Hard complexity directly from Disk Classification
inscope_vinfo_df['Complexity'] = inscope_vinfo_df['Disk Classification'].apply(
    lambda x: 'F5-Hard' if 'F5-Hard' in str(x) else None
)

# Assign complexities based on OS and Disk Size
def classify_complexity(row):
    if row['Complexity'] == 'F5-Hard':  # Skip if already classified as F5-Hard
        return 'Hard'  # Combine F5-Hard into Hard

    if 'oracle' in row['Final OS'].lower():
        return 'Oracle-DBs'
    if any(keyword in row['Final OS'].lower() for keyword in ['mssql', 'sql server', 'microsoft sql', 'ms sql', 'sql database', 'sql']):
        return 'MSSQL-DBs'

    if row['Disk Size TB'] <= 2:
        return 'Easy'
    elif row['Disk Size TB'] <= 5:
        return 'Medium'
    elif row['Disk Size TB'] <= 10:
        return 'Hard'
    else:
        return 'Hard'

inscope_vinfo_df['Complexity'] = inscope_vinfo_df.apply(classify_complexity, axis=1)

# Define the custom sorting order for Complexity
complexity_order = ['Easy', 'Medium', 'Hard', 'Oracle-DBs', 'MSSQL-DBs']
inscope_vinfo_df['Complexity'] = pd.Categorical(
    inscope_vinfo_df['Complexity'],
    categories=complexity_order,
    ordered=True
)

# Sort the DataFrame by Complexity
inscope_vinfo_df = inscope_vinfo_df.sort_values('Complexity')

# Create the OS Support column based on supported_os_list
inscope_vinfo_df['OS Support'] = inscope_vinfo_df['Final OS'].apply(
    lambda os: 'Supported' if any(supported_os in os for supported_os in supported_os_list) else 'Not Supported'
)

# Add Migration Time
inscope_vinfo_df['Migration Time (minutes)'] = inscope_vinfo_df['Disk Size TB'].apply(
    lambda size: ((size * 1024) / 500) * migration_time_per_500gb
)

# Add Post-Migration Troubleshooting Time
pmt_minutes = pmt_hours * 60
inscope_vinfo_df['Total Time (minutes)'] = inscope_vinfo_df['Migration Time (minutes)'] + pmt_minutes

# Summarize by Complexity and OS Support
disk_classification_summary = inscope_vinfo_df.groupby(['Complexity', 'OS Support'], observed=True).agg(
    VM_Count=('VM', 'count'),
    Total_Disk=('Disk Size TB', 'sum'),
    Total_Mig_Time_Minutes=('Total Time (minutes)', 'sum')
).reset_index()

# Include all "OS Support" categories in the summary
for complexity in complexity_order:
    for os_support in ['Supported', 'Not Supported']:
        if not ((disk_classification_summary['Complexity'] == complexity) &
                (disk_classification_summary['OS Support'] == os_support)).any():
            disk_classification_summary = pd.concat([
                disk_classification_summary,
                pd.DataFrame({
                    'Complexity': [complexity],
                    'OS Support': [os_support],
                    'VM_Count': [0],
                    'Total_Disk': [0],
                    'Total_Mig_Time_Minutes': [0]
                })
            ], ignore_index=True)

# Filter out rows where VM_Count is 0
disk_classification_summary = disk_classification_summary[disk_classification_summary['VM_Count'] > 0]

# Add calculated columns for formatted migration time and days
disk_classification_summary['Formatted_Mig_Time'] = disk_classification_summary['Total_Mig_Time_Minutes'].apply(
    lambda minutes: f"{math.ceil(minutes / 30) / 2:,.1f}h"
)
disk_classification_summary['Days_Per_FTEs'] = disk_classification_summary['Total_Mig_Time_Minutes'].apply(
    lambda minutes: math.ceil(minutes / (fte_hours_per_day * 60 * fte_count))
)

# Format VM Count with thousands separator
disk_classification_summary['VM_Count'] = disk_classification_summary['VM_Count'].apply(lambda x: f"{x:,}")

# Format Total Disk (TB) without rounding
disk_classification_summary['Total_Disk'] = disk_classification_summary['Total_Disk'].apply(lambda x: f"{x:,.2f}")

# Add totals row
totals_row = {
    'Complexity': 'Totals',
    'OS Support': '',
    'VM_Count': f"{int(disk_classification_summary['VM_Count'].str.replace(',', '').astype(int).sum()):,}",
    'Total_Disk': f"{float(disk_classification_summary['Total_Disk'].str.replace(',', '').astype(float).sum()):,.2f}",
    'Formatted_Mig_Time': f"{math.ceil(disk_classification_summary['Total_Mig_Time_Minutes'].sum() / 30) / 2:,.1f}h",
    'Days_Per_FTEs': disk_classification_summary['Days_Per_FTEs'].sum()
}
disk_classification_summary = pd.concat(
    [disk_classification_summary, pd.DataFrame([totals_row])],
    ignore_index=True
)

# Print the table
headers = [
    "Complexity", "OS Support", "VM Count", "Total Disk (TB)",
    "Total Migration Time", f"Days ({fte_count} FTEs)"
]
rows = disk_classification_summary[
    ['Complexity', 'OS Support', 'VM_Count', 'Total_Disk', 'Formatted_Mig_Time', 'Days_Per_FTEs']
].values.tolist()

print("\nSummary Table:")
print(custom_table_format_with_totals(headers, rows))

# Display Oracle-DBs and MSSQL-DBs separately
oracle_summary = disk_classification_summary[disk_classification_summary['Complexity'] == 'Oracle-DBs']
mssql_summary = disk_classification_summary[disk_classification_summary['Complexity'] == 'MSSQL-DBs']

print("\nOracle-DBs Summary:")
print(oracle_summary)

print("\nMSSQL-DBs Summary:")
print(mssql_summary)